In [1]:
# Read out CSV file with list of all tickers
nqSdf = spark.read.load('abfss://risk@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/Portfolio/PortfolioCik.csv', 
    format='csv', 
    sep=",",
    header=True)

portfolioData = nqSdf.toPandas()
display(portfolioData)

StatementMeta(FinanceSpk, 56, 1, Finished, Available)

SynapseWidget(Synapse.DataFrame, 0c0d2d48-23ea-4919-a4be-7486b93e31cc)


/opt/spark/python/lib/pyspark.zip/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  'JavaPackage' object is not callable
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.

In [2]:
from azure.storage.blob import BlobServiceClient, ContainerClient
from azure.storage.blob import BlobClient

accountName = "#STORAGE_ACCOUNT_NAME#"
accountKey = "#STORAGE_ACCOUNT_KEY#"
containerName = "risk"
connectionString = "DefaultEndpointsProtocol=https;AccountName=#STORAGE_ACCOUNT_NAME#;AccountKey=#STORAGE_ACCOUNT_KEY#;EndpointSuffix=core.windows.net"

StatementMeta(FinanceSpk, 56, 2, Finished, Available)

## Pulling News Sentiment Data From Bing Api (Azure Cognitive)


In [3]:
import urllib.parse, base64
import json
import requests
import pandas as pd
import datetime
import pytz
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

#Pulling News Sentement Data From Bing Api
search_url = "https://api.bing.microsoft.com/v7.0/news/search"
sentiment_url = 'https://#COGNITIVE_SERVICE_NAME#.cognitiveservices.azure.com/'
sentimentKey = '#COGNITIVE_SERVICE_KEY#'          # Azure Cognitive API Key, replace with your own key
searchKey = '#SEARCH_KEY#'

credential = AzureKeyCredential(sentimentKey)
text_analytics_client = TextAnalyticsClient(endpoint=sentiment_url, credential=credential)

StatementMeta(FinanceSpk, 56, 3, Finished, Available)

ModuleNotFoundError: No module named 'azure.ai'

In [ ]:
#for row in nqDf.rdd.collect():
for index, ticker in portfolioData.iterrows():

    dfObj = pd.DataFrame(columns=['Symbol', 'Name', 'Url', 'Date_Published', 'Description', 'Sentiment', 'Positive_Score', 'Negative_Score', 'Neutral_Score'])

    headers = {'Ocp-Apim-Subscription-Key':searchKey, \
                'Content-type': 'application/json',\
                'Accept': 'application/json'}

    symbol = ticker['Ticker']

    params = urllib.parse.urlencode({
        # Request parameters
        'q': ticker['Ticker'],
        'offset': '0',
        'mkt': 'en-us',
        'freshness': 'Day',
        'count' : 100
    })

    response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()
    #print(json.dumps(search_results['value'], indent=2, sort_keys=True))
    for value in search_results['value']:
        description = value['description']
        url = value['url']
        name = value['name']
        publishedDate = value['datePublished']
    
        documents = ["'" + description + "'"]

        response = text_analytics_client.analyze_sentiment(documents, language="en")
        result = [doc for doc in response if not doc.is_error]
        #print(result)

        for doc in result:
            #print("Overall sentiment: {}".format(doc.sentiment))
            #print("Scores: positive={}; neutral={}; negative={} \n".format(
            #    doc.confidence_scores.positive,
            #    doc.confidence_scores.neutral,
            #    doc.confidence_scores.negative,
            #))
            dfObj = dfObj.append({'Symbol': symbol, 'Name': name, 'Url': url, 'Date_Published': publishedDate, 'Description' : description, 'Sentiment': doc.sentiment, 'Positive_Score':doc.confidence_scores.positive, 'Negative_Score': doc.confidence_scores.negative, 'Neutral_Score': doc.confidence_scores.neutral}, ignore_index=True)
    
    currDate = datetime.datetime.now(pytz.timezone('US/Central'))
    output = dfObj.to_csv (index=False, header=True, encoding = "utf-8")
    blobName = "News/DailyData/" + str(currDate.year) + "/" + str(currDate.strftime("%m")) + "/" + str(currDate.strftime("%d")) + "/" + ticker['Ticker'] + ".csv"
    blob = BlobClient.from_connection_string(conn_str=connectionString, container_name=containerName, blob_name=blobName)
    blob.upload_blob(output)

StatementMeta(, , , Cancelled, )